In [ ]:
import json
import pandas as pd
import numpy as np
from tabulate import tabulate
import os

pd.set_option('display.max_colwidth', None)

### Indicate in the below cell, whether you want to look at all experiments ~ OR~ just a single experiment

In [ ]:
def get_exp(path):
    """ Get Experiment name"""
    path_list = path.split('/')
    exp_name = path_list[-4]
    return exp_name

In [ ]:
experiment_name = 'final_experiments'

In [ ]:
if not experiment_name or experiment_name == 'all':
    exp_folders = [x[0] for x in os.walk('./') if 'output/2021' in x[0]]
else:
    exp_folders = [x[0] for x in os.walk(f'./{experiment_name}') if 'output/2021' in x[0]]

print("Total Experiments: ", len(exp_folders))
print()
print("Sample:")
print(np.array(exp_folders[:5]))


### Load Experiment Results

In [ ]:
result = None
exp_count = 1
for folder in exp_folders:
    files = os.listdir(f'{folder}')
    for file in files:
        if file.endswith(".csv"):
#             print(f'Exp #{exp_count}: {folder}/{file}')
            exp_count += 1
            cur_exp = pd.read_csv(f'{folder}/{file}')
            if result is None:
                cols = list(cur_exp.columns)
                result = pd.DataFrame(columns=cols)
            result = pd.concat([result, cur_exp])

print(result.shape)

### Get experiment names

In [ ]:
# Get experiment names
result['exp'] = result['File'].apply(lambda x: get_exp(x))
result = result.drop(['File'], axis=1)
result.head()

In [ ]:
# Get Each group in sorted order by BLEU score
result = result.groupby(["exp"]).apply(lambda x: x.sort_values(["BLEU"], ascending = False)).reset_index(drop=True)

In [ ]:
exp_names = result['exp'].unique()
exp_names

In [ ]:
for exp in exp_names:
    print(exp)
    top3rows = result[result['exp'] == exp][:3]
#     top3rows = top3rows.drop(['CIDEr','NIST','METEOR','ROUGE_L'], axis=1)
    print(tabulate(top3rows, headers='keys', tablefmt='psql'))
    
    print()
    print()

In [ ]:
result.sort_values(by='BLEU', ascending=False)

In [ ]:
top20 = result.sort_values(by='BLEU', ascending=False)[:20]
top20.head()

In [ ]:
print(top20.drop(["params"], axis=1)[:10].to_latex(index=False))

In [ ]:
result.sort_values(by='version', ascending=False)